In [1]:
import json
import yaml
import boto3

# import sagemaker
import sagemaker.session


from sagemaker.sklearn.estimator import SKLearn
from sagemaker.processing import (
    FrameworkProcessor,
    ProcessingInput,
    ProcessingOutput)
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.steps import ProcessingStep, CacheConfig
from sagemaker.workflow.functions import Join
from sagemaker.workflow.pipeline import Pipeline

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
session = sagemaker.session.Session()
pipe_session = PipelineSession()
region = session.boto_region_name
role = sagemaker.get_execution_role()
account_id = session.account_id()

tags = [
    {"Key": "Test", "Value": "me"}
   ]

In [3]:
config_file = "configs/param_config_simp.yaml"
special_name = "debug"

In [4]:
with open(f"{config_file}", "r",
          encoding="utf-8") as config_file:
    config_params = yaml.safe_load(config_file)

date_ymd = config_params["date_ts"][:10]
pipeline_name = config_params["use_case"] + "Prod"
model_package_group_name = config_params["use_case"].lower() + "Prod"
model_name = model_package_group_name + "-" + config_params["date_ts"]
prod_uri = "s3://{}".format(config_params["bucket"])
model_description = (config_params["target"].split("_")[0]
                     + config_params["filter_value"].capitalize()
                     + str(config_params["day_interval"])
                     + "Day"
                     + date_ymd.replace("-","")
                    )
if special_name is not None:
    model_description = model_description + special_name
process_input_path = "/opt/ml/processing/input"
process_output_path = "/opt/ml/processing/output"

source_dir_path = "source_dir"

pipe_prefix = ["mobility_models", model_description]

data_filepath = ["s3://{}".format(config_params["bucket"]),
                 *pipe_prefix,
                 "data"]

step_counter = 1
step_list = []

step_cache_config = CacheConfig(
    enable_caching=True,
    expire_after=config_params["step_cache_config"])

In [5]:
# s3_resource = boto3.resource('s3')
# s3_resource.Bucket(config_params['bucket']).upload_file('sql_queries/base_query.txt', f"s3://{config_params['bucket']}/{model_description}/sql_queries/base_query.txt")

In [6]:
query_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version=config_params["framework_version"],
    role=role,
    instance_type=config_params["instance_type_processing"],
    instance_count=config_params["instance_count_processing"],
    sagemaker_session=pipe_session,
    base_job_name="data_query")

query_list = []

In [7]:
# Query one table
# Set arguments
table = "WEB_DATA"
arguments = [
    "--table", table,
    "--input-path", process_input_path,
    "--output_path", process_output_path,
    "--db-type", "mysql",
    "--dbname", "machinelearning",
    "--folder-date", date_ymd,
    "--interval", config_params["day_interval"]
]

query_args = query_processor.run(
    code="feat_query.py",
    source_dir=source_dir_path,
    inputs=[
        ProcessingInput(
            source=Join(
                on="/",
                values=[
                    "s3://".format(config_params['bucket']),
                    model_description,
                    "sql_queries",
                    "base_query.txt"]),
            destination=process_input_path,
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="features",
            source=process_output_path,
            destination=Join(
                on="/",
                values=[
                    *data_filepath,
                    "features"])
        )
    ],
    arguments=arguments
)

query_dbs = ProcessingStep(
    name="{}-{}".format(step_counter, table),
    step_args=query_args,
    cache_config=step_cache_config
)
step_list.append(query_dbs)
query_list.append(query_dbs)

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [8]:
# Create Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    # parameters=param_list,
    steps=step_list,
    sagemaker_session=pipe_session)

with open("pipeline_definition_simp.json", "w", encoding="utf-8") as def_file:
    json.dump(json.loads(pipeline.definition()), def_file, indent=4)

# Upload Pipeline and Start Exedution

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(
    execution_display_name=model_description)

[04/23/25 23:11:01] INFO     Uploaded source_dir to                                              ]8;id=537423;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=360244;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-888962879387/DebugProd/code/b88bcd50813de7                   
                             95c4852e161640cd0eb52bd53429ab911edacea56ad7f9b46a/sourcedir.tar.gz                   

[04/23/25 23:11:02] INFO     runproc.sh uploaded to                                              ]8;id=548661;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=869251;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-888962879387/DebugProd/code/7064c6f3bf668d                   
                             bd59829e69e684badc12291a73e86c751fc9c32a8d8cc74f10/runproc.sh                         

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=81111;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=455038;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    INFO     Uploaded source_dir to                                              ]8;id=419088;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=221301;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-888962879387/DebugProd/code/b88bcd50813de7                   
                             95c4852e161640cd0eb52bd53429ab911edacea56ad7f9b46a/sourcedir.tar.gz                   

                    INFO     runproc.sh uploaded to                                              ]8;id=385198;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=721317;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-888962879387/DebugProd/code/7064c6f3bf668d                   
                             bd59829e69e684badc12291a73e86c751fc9c32a8d8cc74f10/runproc.sh                         

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=777271;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=78406;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11                                                                                             │
│   12 # Upload Pipeline and Start Exedution                                                       │
│   13                                                                                             │
│ ❱ 14 pipeline.upsert(role_arn=role, tags=tags)                                                   │
│   15                                                                                             │
│   16 pipeline.start(                                                                             │
│   17 │   execution_display_name=model_description)                                               │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline.py:297 in upsert             │
│                                                                                                  │
│    294 │   │   │   error_code = ce.response["Error"]["Code"]                                     │
│    295 │   │   │   error_message = ce.response["Error"]["Message"]                               │
│    296 │   │   │   if not (error_code == "ValidationException" and "already exists" in error_me  │
│ ❱  297 │   │   │   │   raise ce                                                                  │
│    298 │   │   │   # already exists                                                              │
│    299 │   │   │   response = self.update(role_arn, description, parallelism_config=parallelism  │
│    300 │   │   │   # add new tags to existing resource                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline.py:292 in upsert             │
│                                                                                                  │
│    289 │   │   │   # after fetching the config.                                                  │
│    290 │   │   │   raise ValueError("An AWS IAM role is required to create or update a Pipeline  │
│    291 │   │   try:                                                                              │
│ ❱  292 │   │   │   response = self.create(role_arn, description, tags, parallelism_config)       │
│    293 │   │   except ClientError as ce:                                                         │
│    294 │   │   │   error_code = ce.response["Error"]["Code"]                                     │
│    295 │   │   │   error_message = ce.response["Error"]["Message"]                               │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline.py:169 in create             │
│                                                                                                  │
│    166 │   │   │   kwargs,                                                                       │
│    167 │   │   │   Tags=tags,                                                                    │
│    168 │   │   )                                                                                 │
│ ❱  169 │   │   return self.sagemaker_session.sagemaker_client.create_pipeline(**kwargs)          │
│    170 │                                                                                         │
│    171 │   def _create_args(                                                                     │
│    172 │   │   self, role_arn: str, description: str, paral